<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_7B_Instruct_4bit_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-7B-Instruct'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [6]:
'''model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

'model = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  torch_dtype = torch.float16,\n  trust_remote_code = True\n).to(device) #'

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic condition characterized by high blood sugar levels. The symptoms and treatments can vary depending on the type of diabetes (Type 1, Type 2, or gestational diabetes) and the severity of the condition. Here's a general overview:

#### Symptoms of Diabetes:

1. **Increased thirst and frequent urination**: This is often one of the first signs of diabetes.
2. **Extreme hunger**: Despite eating more, you may feel very hungry.
3. **Unexplained weight loss**: This is more common in Type 1 diabetes.
4. **Fatigue**: Feeling tired and weak.
5. **Blurred vision**: High blood sugar can affect your vision.
6. **Slow-healing sores**: High blood sugar can slow down the healing process.
7. **Frequent infections**: You may get more infections, especially of the skin, bladder, and vaginal areas.
8. **Numbness or tingling in the hands or feet**: This can be a sign of nerve damage (neuropathy).

#### Treatmen

In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic condition characterized by high blood sugar levels. The symptoms of diabetes can vary depending on the type of diabetes, but common symptoms include frequent urination, excessive thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow wound healing. 

There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune condition where the body does not produce insulin, while Type 2 diabetes is characterized by the body's resistance to insulin or the inability to produce enough insulin. 

Treatment for diabetes typically involves a combination of lifestyle changes, medication, and sometimes insulin therapy. Lifestyle changes include a healthy diet, regular exercise, and maintaining a healthy weight. Medications may include oral medications to help control blood sugar levels or insulin injections for Type 1 diabetes. 

In addition to t

In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic condition that affects how the body uses glucose (a type of sugar) for energy. There are two main types of diabetes: Type 1 and Type 2. The symptoms and treatments can vary depending on the type of diabetes, but there are some common symptoms and treatments that apply to both.

### Common Symptoms:

1. **Increased thirst and frequent urination**: People with diabetes often feel very thirsty and need to urinate more frequently, especially at night.
2. **Extreme hunger**: Despite eating more, people with diabetes may feel very hungry.
3. **Unexplained weight loss**: In Type 1 diabetes, people may lose weight without trying, even though they are eating more.
4. **Fatigue**: Feeling very tired or weak is a common symptom of diabetes.
5. **Blurred vision**: High blood sugar levels can cause vision problems.
6. **Slow-healing sores or frequent infections**: People wi

In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : The main symptoms of diabetes include:

1. **Increased thirst (polydipsia)**
2. **Frequent urination (polyuria)**
3. **Extreme hunger (polyphagia)**
4. **Unexplained weight loss**
5. **Fatigue**
6. **Blurred vision**
7. **Slow-healing sores or infections**
8. **Numbness or tingling in the hands or feet**
9. **Dry, itchy skin**
11. **Fruity-smelling breath**

### Standard Treatments for Diabetes:

1. **Medications:**
   - **Oral medications:** These include metformin, sulfonylureas, thiazolidinediones, and others. They help control blood sugar levels by increasing insulin sensitivity or stimulating insulin production.
   - **Insulin:** For people with type 1 diabetes and some with type 2 diabetes, insulin injections or insulin pump therapy are necessary to manage blood sugar levels.

2. **Dietary Changes:**
   - **Balanced meals:** Eating a diet that is low in sugar and high in fiber, wh

In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Type 1 and Type 2 diabetes are two common forms of diabetes, but they have different causes, symptoms, and treatments. Here's a detailed list for each:

#### Type 1 Diabetes:
**Symptoms:**
- Frequent urination
- Extreme thirst
- Unexplained weight loss
- Extreme hunger
- Blurred vision
- Fatigue
- Slow-healing wounds
- Frequent infections (especially yeast infections or urinary tract infections)

**Treatments:**
- Insulin therapy (usually multiple daily injections or an insulin pump)
- Blood glucose monitoring
- Proper nutrition and meal planning
- Regular physical activity
- Stress management
- Education and support from healthcare professionals

#### Type 2 Diabetes:
**Symptoms:**
- Frequent urination
- Extreme thirst
- Unexplained weight loss
- Extreme hunger
- Blurred vision
- Fatigue
- Slow-healing wounds
- Frequent infections (especially yeast infections or urinary tract infections)
-

In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 and Type 2 diabetes are two distinct forms of diabetes mellitus, each with its own set of symptoms, diagnostic criteria, and treatment options. Here's a detailed overview for healthcare providers:

#### Symptoms:
**Type 1 Diabetes:**
- **Polyuria (frequent urination)**
- **Polydipsia (increased thirst)**
- **Polyphagia (increased hunger)**
- **Unintended weight loss**
- **Fatigue**
- **Blurred vision**
- **Ketoacidosis (rare but serious, characterized by nausea, vomiting, abdominal pain, and fruity breath)**

**Type 2 Diabetes:**
- **Polyuria (frequent urination)**
- **Polydipsia (increased thirst)**
- **Polyphagia (increased hunger)**
- **Unintended weight loss**
- **Fatigue**
- **Blurred vision**
- **Slow-healing wounds**
- **Recurrent infections (especially of the skin, urinary tract, and vaginal area)*

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : Diabetes is a group of metabolic disorders characterized by high blood glucose levels over an extended period. It results from defects in insulin production, insulin action, or both. There are three main types of diabetes: prediabetes, Type 1, and Type 2. Each type has distinct symptoms, diagnostic markers, and treatments.

### Prediabetes
**Symptoms:**
- Prediabetes often has no noticeable symptoms, making it difficult to detect without a blood test.
- Some people may experience increased thirst, frequent urination, or fatigue.

**Diagnostic Markers:**
- Fasting Plasma Glucose (FPG) between 100-125 mg/dL
- Oral Glucose Tolerance Test (OGTT) 2-hour plasma glucose between 140-199 mg/dL
- Hemoglobin A1C (HbA1C) between 5.7-6.4%

**Treatments:**
- Lifestyle changes: diet, ex

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant : Diabetes is a group of metabolic disorders characterized by high blood glucose levels over an extended period. It results from defects in insulin production, insulin action, or both. There are two main types of diabetes: Type 1 and Type 2.

### Symptoms
**Type 1 Diabetes:**
- **Polyuria (frequent urination)**
- **Polydipsia (increased thirst)**
- **Polyphagia (increased hunger)**
- **Unintended weight loss**
- **Fatigue**
- **Blurred vision**
- **Frequent infections (especially skin and vaginal)**
- **Ketoacidosis (rare but serious, characterized by nausea, vomiting, and abdominal pain)**

**Type 2 Diabetes:**
- **Polyuria (frequent urination)**
- **Polydipsia (increased thirst)**
- **Polyphagia (increased hunger)**
- **Unintended weight loss**
- *

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant : Diabetes is a chronic condition characterized by high blood glucose levels due to defects in insulin production, insulin action, or both. It is a complex disease with various types, the most common being Type 1 and Type 2 diabetes. Understanding the symptoms, diagnostic criteria, and treatment options for both types, as well as the risks and complications associated with diabetes, is crucial for effective management.

### Symptoms of Diabetes

**Type 1 Diabetes:**
- Frequent urination
- Extreme thirst
- Unexplained weight loss
- Fatigue
- Blurred vision
- Increased hunger
- Slow-healing wounds
- Numbness or tingling in the 

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood glucose levels due to defects in insulin production, insulin action, or both. It is a complex condition with two main types: Type 1 and Type 2. Understanding the pathophysiology, diagnostic criteria, treatment approaches, and complications of diabetes is crucial for effective management.

#### Pathophysiology

**Type 1 Diabetes:**
- **Pathophysiology:** Type 1 diabetes results from autoimmune destruction of pancreatic beta c